In [3]:
import pandas as pd
import numpy as np
from cleaning import sessioninfo, scores, create
import pickle
from sklearn.model_selection import train_test_split
import project_functions
from project_functions import decompose, drop_stubs, make_data_rectangular
from sklearn.linear_model import LinearRegression
import pymc as pm
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import math
import torch

#TODO: Remove project_functions, cleaning, and CAPS.py and other stuff that isn't used

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
REPLACE_VAL = 0.5
#Gamme distributions can't actually take 0 values with nonzero probability, so Bayesian credible intervals require that all 0 errors be replaced with something else

# Data Processing

## Gathering and processing raw data

In [ ]:
#Set up options for data cleaning
# besides data path root folders, copied from code by Seth Peacock, who wrote the cleaning code (except for a few tweaks)
data_path_old = [".."]
data_path_new = ["..", "2018-2023"]
recent_dict = {'Within the last 2 weeks': 1, 'Within the last month': 1,
        'Within the last year': 1, 'Within the last 1-5 years': 0,
        'More than 5 years ago': 0, 'Never': 0, '<No Response>': 0}

session_arg = dict(unique_therapist=True, # unique therapist
                   intake_dif=False,
                   min_appoint=1,
                   max_diff_between_apps=180)

match_arg = dict(recent_dict=recent_dict)

imput_arg = dict(na_cutoff=1,
                 n_components=10)

scores_arg = dict(days_before=30, 
                  days_after=7)

filter_arg = dict(cutoff_hi=180, 
                  cutoff_low=0)

### Train and Validation

In [ ]:
#Get the non-hidden clients
app_hid, patient_info, scores_df = create.get_old_data(data_path_new, data_path_old, data_path_old, hide=True, dataset="old")

master_df_1, app_session = sessioninfo.make_session(appoint_data=app_hid, **session_arg)

In [ ]:
#Link appointment data to patient information
master_df_2 = sessioninfo.match_patient_info(master_df=master_df_1, 
                                                 patient_data=patient_info,
                                                 **match_arg)

In [ ]:
master_df_2.to_csv('master_df_2_cu002_thesis.csv', index=False)

In [ ]:
#Link appointment data to OQ score data
app_oq = scores.oq_score(app_session=app_session, scores=scores_df, **scores_arg)

In [ ]:
app_oq.to_csv('appointment_oq_cu002_thesis.csv', index=False)

### Test

In [ ]:
#As above, but with the hidden clients
app_hid, patient_info, scores_df = create.get_old_data(data_path_new, data_path_old, data_path_old, hide=False, dataset="old")

master_df_1, app_session = sessioninfo.make_session(appoint_data=app_hid, **session_arg)

In [ ]:
master_df_2 = sessioninfo.match_patient_info(master_df=master_df_1, 
                                                 patient_data=patient_info,
                                                 **match_arg)

In [ ]:
master_df_2.to_csv('master_df_2_cu003_thesis_hiddenClients.csv', index=False)

In [ ]:
app_oq = scores.oq_score(app_session=app_session, scores=scores_df, **scores_arg)

In [ ]:
app_oq.to_csv('appointment_oq_cu003_thesis_hiddenClients.csv', index=False)

## Make usable for time series analysis

In [ ]:
appointments_df = pd.read_csv('appointment_oq_cu002_thesis.csv')
appointments_df.dropna(inplace=True, subset=['ClientID', 'SessionID', 'CurrentScore', 'Date'])

In [ ]:
#For redundancy, exclude clients on the list of hidden clients
hidden_clients_list = []
with open('../ClientsHiddenAway', 'r') as file:
    for line in file:
        hidden_clients_list.append(int(line.strip()))

In [ ]:
non_hidden_clients = set(appointments_df['ClientID'].unique())
print(len(list(non_hidden_clients)))

#Create sets of clients for training and validation
train_clients, val_clients = train_test_split(list(non_hidden_clients), test_size=0.25, random_state = 404)

In [ ]:
assert len(set(train_clients).intersection(set(val_clients))) == 0

#Save train and val clients for future reference
with open('local_train_clients.txt', 'w') as file:
    for client in train_clients:
        file.write(str(client)+'\n')

with open('local_val_clients.txt', 'w') as file:
    for client in val_clients:
        file.write(str(client)+'\n')

In [ ]:
#Make sure that no clients weren't included in part of the split

with open('local_train_clients.txt', 'r') as file:
    train_lines = file.readlines()
train_clients = [int(line.split()[0]) for line in train_lines]

with open('local_val_clients.txt', 'r') as file:
    val_lines = file.readlines()
val_clients = [int(line.split()[0]) for line in val_lines]

all_sorted_clients = set(train_clients).union(set(val_clients)).union(set(hidden_clients_list))

new_clients = []
num_old = 0
for client_ID in appointments_df['ClientID']:
    if client_ID not in all_sorted_clients and client_ID not in new_clients:
        new_clients.append(client_ID)
    else:
        num_old += 1
print(len(new_clients))
if len(new_clients) > 0:
    new_train_clients, new_val_clients = train_test_split(new_clients, test_size=0.25, random_state = 404)
    print(len(new_train_clients))
    print(len(new_val_clients))
    try:
        assert len(set(new_train_clients).intersection(set(new_val_clients)))==0
    except:
        print(set(new_train_clients).intersection(set(new_val_clients)))
        raise
    train_clients = train_clients + new_train_clients
    val_clients = val_clients + new_val_clients

In [ ]:
assert len(set(train_clients).intersection(set(val_clients))) == 0


#Save updated train and val clients for future reference
if len(new_clients) > 0:
    with open('local_train_clients.txt', 'w') as file:
        for client in train_clients:
            file.write(str(client)+'\n')

    with open('local_val_clients.txt', 'w') as file:
        for client in val_clients:
            file.write(str(client)+'\n')

In [ ]:
#Make appointments_df into something we can get time series out of
appointments_df.rename({'SessionID':'Session', 'CurrentScore': 'OQ'}, inplace=True, axis=1)
appointments_df = appointments_df[['ClientID', 'Session', 'OQ', 'Date']].drop_duplicates()
appointments_df['Session'] = appointments_df['Session'].astype(int)

appointments_df['Date'] = pd.to_datetime(appointments_df['Date'])
    # the argument format='ISO8601' was necessary on some machines

appointments_df.sort_values(['Session', 'Date'], inplace=True)

train_appts = appointments_df.loc[appointments_df['ClientID'].isin(train_clients)].dropna()
val_appts = appointments_df.loc[appointments_df['ClientID'].isin(val_clients)].dropna()

#This is a list of lists. Each of the inside lists is a time series.
OQ_lists = []
for df in [train_appts, val_appts]:
    OQ_lists.append([list(df[df['Session'] == session]['OQ'].values) for session in df['Session'].unique()])

train_OQ_list, val_OQ_list = OQ_lists

In [ ]:
#Save the timeseries data to a pickled file
with open('OQ_lists_new.pkl', 'wb') as file:
    pickle.Pickler(file=file).dump(OQ_lists)

In [ ]:
#Now do the test data
appointments_df_test = pd.read_csv('appointment_oq_cu003_thesis_hiddenClients.csv')
appointments_df_test.dropna(inplace=True, subset=['ClientID', 'SessionID', 'CurrentScore', 'Date'])

all_sorted_clients = set(train_clients).union(set(val_clients)).union(set(hidden_clients_list))
for client_ID in appointments_df_test['ClientID']:
    assert client_ID in hidden_clients_list

appointments_df_test.rename({'SessionID':'Session', 'CurrentScore': 'OQ'}, inplace=True, axis=1)
appointments_df_test = appointments_df_test[['ClientID', 'Session', 'OQ', 'Date']].drop_duplicates()
appointments_df_test['Session'] = appointments_df_test['Session'].astype(int)

appointments_df_test['Date'] = pd.to_datetime(appointments_df_test['Date'])

appointments_df_test.sort_values(['Session', 'Date'], inplace=True)

test_OQ_list = [list(appointments_df_test[appointments_df_test['Session'] == session]['OQ'].values) for session in appointments_df_test['Session'].unique()]

In [ ]:
with open('test_list_new.pkl', 'wb') as file:
    pickle.Pickler(file=file).dump(test_OQ_list)

In [ ]:
#Link session numbers to observations for test data
test_sessions_list = appointments_df_test['Session'].unique()
test_OQ_list = [list(appointments_df_test[appointments_df_test['Session'] == session]['OQ'].values) for session in test_sessions_list]

test_rows_list = []

for i, OQ_list in enumerate(test_OQ_list):
    truncations, nexts = decompose(OQ_list)
    for j in range(len(truncations)):
        test_rows_list.append({
            'Session': test_sessions_list[i],
            'Previous': truncations[j],
            'Next': nexts[j],
            'Num_Steps': len(truncations)
        })

test_data_df = pd.DataFrame(test_rows_list)
with open('test_data_new.pkl', 'wb') as file:
    pickle.Pickler(file=file).dump(test_data_df)

# Generation of a Prior

## Load data

In [ ]:
#Load cleaned data
with open('OQ_lists_new.pkl', 'rb') as file:
    OQ_lists_new = pickle.Unpickler(file).load()
train_OQ_list_new, val_OQ_list_new = OQ_lists_new

# Drop stubs
train_OQ_list_new = drop_stubs(train_OQ_list_new)
val_OQ_list_new = drop_stubs(val_OQ_list_new)


val_prevs_list_new = []
val_next_list_new = []
for timeseries in val_OQ_list_new:
    prevs, nexts = decompose(timeseries)
    for i in range(len(prevs)):
        val_prevs_list_new.append(prevs[i])
        val_next_list_new.append(nexts[i])
val_next_list_new = np.array(val_next_list_new)

## Make predictions by hand

In [ ]:
np.random.seed(42)
indices_to_sample_new = np.random.choice(len(val_prevs_list_new), replace=False, size=50)

time_series_list = [val_prevs_list_new[indices_to_sample_new[i]] for i in range(50)]

The actual prediction process is omitted to protect data privacy. The author observed and plotted each of the 50 validation-set time series, recorded his guess for the next value, and after all predictions were made, compared his predictions to the true values to find the following errors.

In [ ]:
abs_errors = np.array([7,14,16,5,5,9,16,1,6,1,10,11,9,1,17,9,12,12,5,22,4,13,10,2,14,6,1,19,13,3,13,22,7,2,4,1,2,7,8,42,1,6,6,11,5,2,12,1,15,REPLACE_VAL])
#Last prediction, 0, replaced

## Estimate parameters of priors

In [ ]:
#Generate bootstrap samples to measure mean and variance empirically
def get_bootstrap_sample(bootstrap_n):
    indices = np.random.choice(len(abs_errors), size=bootstrap_n, replace=True)
    return abs_errors[indices]

means = []
vars = []
np.random.seed(42)
for i in range(30):
    sample = get_bootstrap_sample(20)
    means.append(np.mean(sample))
    vars.append(np.var(sample))

In [ ]:
means_mean = np.mean(means)
means_var = np.var(means)

#Variance will have a gamma prior, so use the mean and variance of the variance samples to derive alpha and beta (rate)
vars_mean = np.mean(vars)
vars_var = np.var(vars)
vars_b = vars_mean/vars_var
vars_a = vars_mean*vars_b

In [ ]:
print(means_mean)
print(means_var)
print(vars_a)
print(vars_b)

# Baselines

## Load data

In [ ]:
#Load cleaned test data

with open('test_list_new.pkl', 'rb') as file:
    test_OQ_list_new = pickle.Unpickler(file).load()

test_OQ_list_new = drop_stubs(test_OQ_list_new)

test_prevs_list = []
test_next_list = []
for timeseries in test_OQ_list_new:
    prevs, nexts = decompose(timeseries)
    for i in range(len(prevs)):
        test_prevs_list.append(prevs[i])
        test_next_list.append(nexts[i])
true_vals = np.array(test_next_list)

## Define predictor classes

In [ ]:
class GlobalMeanGuesser():
    def fit(self, train_OQ_list):
        self.global_mean = np.mean(np.concatenate(train_OQ_list))
    def predict(self, oq):
        return self.global_mean
    def predict_full_data(self, data):
        return np.full(len(data), self.global_mean)
    
class PersonalMeanGuesser():
    def predict(self, oq):
        return np.mean(oq)

class FirstScoreGuesser():
    def predict(self, oq):
        return oq[0]

class LastScoreGuesser():
    def predict(self, oq):
        return oq[-1]

class DumbLineGuesser(): #Guess that they'll follow the average slope of the series; not line of best fit, just last-first
    def predict(self, oq):
        if len(oq) == 1:
            return oq[0]
        else:
            return oq[-1]+(oq[-1]-oq[0])/(len(oq)-1)
    
class SmartLineGuesser(): #Get a line of best fit
    def predict(self, oq):
        linreg = LinearRegression()
        linreg.fit(np.expand_dims(np.arange(len(oq)), 0).reshape(-1,1), oq)
        return linreg.predict(np.expand_dims(np.array([len(oq)]),0))[0]

class GlobalMedianGuesser():
    def fit(self, train_OQ_list):
        self.global_median = np.median(np.concatenate(train_OQ_list))
    def predict(self, oq):
        return self.global_median
    def predict_full_data(self, data):
        return np.full(len(data), self.global_median)

class PersonalMedianGuesser():
    def predict(self, oq):
        return np.median(oq)

class LastMeanMidpointGuesser():
    def predict(self,oq):
        return np.mean([np.mean(oq), oq[-1]])
    
class LastMedianMidpointGuesser():
    def predict(self,oq):
        return np.mean([np.median(oq), oq[-1]])

class LastDumblineMidpointGuesser():
    def predict(self, oq):
        if len(oq) == 1:
            return oq[0]
        else:
            return np.mean([oq[-1]+(oq[-1]-oq[0])/(len(oq)-1), oq[-1]])

class LastSmartlineMidpointGuesser():
    def predict(self, oq):
        linreg = LinearRegression()
        linreg.fit(np.expand_dims(np.arange(len(oq)), 0).reshape(-1,1), oq)
        return np.mean([linreg.predict(np.expand_dims(np.array([len(oq)]),0))[0], oq[-1]])

class LastSmartlineMeanFusionGuesser():
    def predict(self,oq):
        linreg = LinearRegression()
        linreg.fit(np.expand_dims(np.arange(len(oq)), 0).reshape(-1,1), oq)
        return np.mean([linreg.predict(np.expand_dims(np.array([len(oq)]),0))[0], oq[-1], np.mean(oq)])

## Get results

### Prediction errors

In [ ]:
name_error_dict = {} # To match a description of each baseline with its list of absolute errors

global_mean_guesser = GlobalMeanGuesser()
global_mean_guesser.fit(train_OQ_list_new)
global_mean_abs_errors = np.abs(np.array([global_mean_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Global mean'] = global_mean_abs_errors

personal_mean_guesser = PersonalMeanGuesser()
personal_mean_abs_errors = np.abs(np.array([personal_mean_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Personal mean'] = personal_mean_abs_errors

first_score_guesser = FirstScoreGuesser()
first_score_abs_errors = np.abs(np.array([first_score_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['First score'] = first_score_abs_errors

last_score_guesser = LastScoreGuesser()
last_score_abs_errors = np.abs(np.array([last_score_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Last score'] = last_score_abs_errors

dumb_line_guesser = DumbLineGuesser()
dumb_line_abs_errors = np.abs(np.array([dumb_line_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Dumb line'] = dumb_line_abs_errors

smart_line_guesser = SmartLineGuesser()
smart_line_abs_errors = np.abs(np.array([smart_line_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Smart line'] = smart_line_abs_errors

global_median_guesser = GlobalMedianGuesser()
global_median_abs_errors = np.abs(np.array([global_median_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Global median'] = global_median_abs_errors

personal_median_guesser = PersonalMedianGuesser()
personal_median_abs_errors = np.abs(np.array([personal_median_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Personal median'] = personal_median_abs_errors

last_mean_midpoint_guesser = LastMeanMidpointGuesser()
last_mean_midpoint_abs_errors = np.abs(np.array([last_mean_midpoint_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Last-mean midpoint'] = last_mean_midpoint_abs_errors

last_median_midpoint_guesser = LastMedianMidpointGuesser()
last_median_midpoint_abs_errors = np.abs(np.array([last_median_midpoint_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Last-median midpoint'] = last_median_midpoint_abs_errors

last_dumbline_midpoint_guesser = LastDumblineMidpointGuesser()
last_dumbline_midpoint_abs_errors = np.abs(np.array([last_dumbline_midpoint_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Last-dumbline midpoint'] = last_dumbline_midpoint_abs_errors

last_smartline_midpoint_guesser = LastSmartlineMidpointGuesser()
last_smartline_midpoint_abs_errors = np.abs(np.array([last_smartline_midpoint_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Last-smartline midpoint'] = last_smartline_midpoint_abs_errors

last_smartline_mean_fusion_guesser = LastSmartlineMeanFusionGuesser()
last_smartline_mean_fusion_abs_errors = np.abs(np.array([last_smartline_mean_fusion_guesser.predict(test_data) for test_data in test_prevs_list])-true_vals)
name_error_dict['Last-smartline-mean fusion'] = last_smartline_mean_fusion_abs_errors

### Mean Squared Error

In [ ]:
for key in name_error_dict.keys():
    print(f"{key}: MSE = {np.mean(name_error_dict[key]**2)}")

### Mean Absolute Error with credible intervals

In [ ]:
output_dict = {}
for key in name_error_dict.keys():
    abs_errors = name_error_dict[key]
    mae_estimate = np.mean(abs_errors)
    nonzero_errors = abs_errors.copy()
    nonzero_errors[nonzero_errors==0]=REPLACE_VAL

    with pm.Model() as model:
        #Put priors on the mean and variance, then derive the corresponding gamma distribution and do inference
        mean = pm.Normal('mean', means_mean, sigma=np.sqrt(means_var))
        var = pm.Gamma('var', alpha=vars_a, beta=vars_b)
        y = pm.Gamma('absolute error', alpha=mean**2/var, beta=mean/var, observed=nonzero_errors)
        trace=pm.sample(25000)
    
    all_mean_draws = np.concatenate(np.array(trace['posterior']['mean']))
    cutoff_idx = int(.025*len(all_mean_draws)) #Find where to pull for the 2.5th percentile
    lower_bound = np.sort(all_mean_draws)[cutoff_idx]
    upper_bound = np.sort(all_mean_draws)[-cutoff_idx-1]
    output_dict[key] = f"{key}: MAE = {mae_estimate} ({lower_bound}-{upper_bound})"

In [ ]:
for key in output_dict.keys():
    print(output_dict[key])

# Linear Regression

## Load data

In [ ]:
overall_mean_OQ_train = np.mean(np.concatenate(train_OQ_list_new))
overall_stdev_OQ_train = np.std(np.concatenate(train_OQ_list_new))
#IMPORTANT! These have specific names because WidthRidge (below) relies on locals with those names.

## Define class

In [ ]:
#A wrapper that allows cross-validation grid search of a Ridge model with dataset width as a parameter
class WidthRidge(Ridge):

    def __init__(self, df_width=1, impute_val=None, use_intake=False, restrict_to_intake=False, exclude_cols = ['Timeseries', 'PatientID', 'SessionID', 'StartDate', 'EndDate', 'NumOfAttended', 'EndDate_intake', 'LastAppShowed', 'AttendRate', 'notedate', 'DateDifference', 'TherapistID', 'NursingOrLaw', 'ReligiousMinority', 'Crisis'], alpha=1.0, fit_intercept=True, copy_X=True, max_iter=None, tol=0.0001, solver='auto', positive=False, random_state=None):

        super().__init__(alpha=alpha, fit_intercept=fit_intercept, copy_X=copy_X, max_iter=max_iter, tol=tol, solver=solver, positive=positive, random_state=random_state)
        self.df_width = df_width
        self.impute_val = impute_val
        self.use_intake = use_intake #Actually train on intake info
        self.restrict_to_intake = restrict_to_intake or self.use_intake #Only use rows that had intake data
        self.exclude_cols = exclude_cols

    def fit(self, X, y, sample_weight=None): #y is a necessary dummy
        #Assume that X is actually a set of variable-length time series, which we turn into data and labels
        if self.restrict_to_intake:
            known_X = X[[col for col in X.columns if col not in self.exclude_cols]]
            self.train_feature_means = np.array(np.mean(known_X, axis=0))
            self.train_feature_stdevs = np.array(np.std(known_X, axis=0))
            timeseries_list_train = [timeseries for timeseries in X['Timeseries'] if len(timeseries) > 1]
            num_rows_per_series_train = [len(timeseries)-1 for timeseries in timeseries_list_train]
            index_list_train = []
            for i, count in enumerate(num_rows_per_series_train):
                index_list_train = index_list_train+[i]*count
            mean = np.mean(np.concatenate(timeseries_list_train))
            stdev = np.mean(np.concatenate(timeseries_list_train))
        else:
            mean = np.mean(np.concatenate(X))

        if self.impute_val is None:
            self.impute_val = mean
        
        if self.restrict_to_intake:
            #normalize
            data_train, labels_train = make_data_rectangular(timeseries_list_train, df_width=self.df_width, impute_val=self.impute_val)
            data_train = (data_train - mean)/stdev
            assert len(data_train) == len(index_list_train)
            if self.use_intake:
                intake_array_train = np.array(known_X)[index_list_train,:]
                intake_array_train = (intake_array_train - self.train_feature_means)/ self.train_feature_stdevs
                assert len(intake_array_train) == len(data_train)
                full_array_train = np.hstack([intake_array_train, data_train])
                super().fit(full_array_train, labels_train)
            else:
                super().fit(data_train, labels_train)
        else:
            train_data, train_labels = make_data_rectangular(X, df_width=self.df_width, impute_val=self.impute_val)
            super().fit(train_data, train_labels, sample_weight=sample_weight)
            
    def predict(self, X):
        #Note: If things aren't all already df_width in width,
        #   this will produce a different number of outputs than rows in X!!
        
        if self.use_intake: #If this is true, restrict_to_intake must be too
            known_X = X[[col for col in X.columns if col not in self.exclude_cols]]
            timeseries_list_val = [timeseries for timeseries in X['Timeseries'] if len(timeseries) > 1]
            num_rows_per_series_val = [len(timeseries)-1 for timeseries in timeseries_list_val]
            index_list_val = []
            for i, count in enumerate(num_rows_per_series_val):
                index_list_val = index_list_val+[i]*count

            data_val, labels_val = make_data_rectangular(timeseries_list_val, self.df_width)
            data_val = (data_val - overall_mean_OQ_train)/overall_stdev_OQ_train
            assert len(data_val) == len(index_list_val)
            intake_array_val = np.array(known_X)[index_list_val,:]
            intake_array_val = (intake_array_val - self.train_feature_means)/self.train_feature_stdevs
            assert len(intake_array_val) == len(data_val)
            full_array_val = np.hstack([intake_array_val, data_val])
            return super().predict(full_array_val)

        else:
            val_data, val_labels = make_data_rectangular(X, df_width=self.df_width, impute_val=self.impute_val)
            return super().predict(val_data)
        
    def score(self, X, y, sample_weight = None):
        #Overriding to use MSE (or rather, negative MSE)
        if self.use_intake: #If this is true, restrict_to_intake must be too
            known_X = X[[col for col in X.columns if col not in self.exclude_cols]]
            timeseries_list_val = [timeseries for timeseries in X['Timeseries'] if len(timeseries) > 1]
            num_rows_per_series_val = [len(timeseries)-1 for timeseries in timeseries_list_val]
            index_list_val = []
            for i, count in enumerate(num_rows_per_series_val):
                index_list_val = index_list_val+[i]*count

            data_val, val_labels = make_data_rectangular(timeseries_list_val, self.df_width)
            data_val = (data_val - overall_mean_OQ_train)/overall_stdev_OQ_train
            assert len(data_val) == len(index_list_val)
            intake_array_val = np.array(known_X)[index_list_val,:]
            intake_array_val = (intake_array_val - self.train_feature_means)/self.train_feature_stdevs
            assert len(intake_array_val) == len(data_val)
            full_array_val = np.hstack([intake_array_val, data_val])
            predictions = super().predict(full_array_val)

        else:
            val_data, val_labels = make_data_rectangular(X, df_width=self.df_width, impute_val=self.impute_val)
            predictions = super().predict(val_data)
        return -1*mean_squared_error(predictions, val_labels)

## Grid search

In [ ]:
ridge_model = WidthRidge()

param_grid = {'alpha':[0.001, 0.01, 0.1, 1, 10, 100],
              'df_width':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20],
              'tol':[0.00001, 0.0001, 0.001],
              'solver':['auto','svd','cholesky','lsqr','saga'],
              'random_state':[64]}

grid_searcher = GridSearchCV(ridge_model, param_grid, cv=3, verbose=2)

grid_searcher.fit(train_OQ_list_new, np.zeros(len(train_OQ_list_new)))

In [ ]:
print(grid_searcher.best_params_)
params_dict = {key:grid_searcher.best_params_[key] for key in grid_searcher.best_params_.keys() if key != 'df_width'}
best_width = grid_searcher.best_params_['df_width']

## Results

### Errors, MSE, MAE

In [ ]:
ridge_model = Ridge(**params_dict)

train_data, train_labels = make_data_rectangular(train_OQ_list_new, df_width=best_width, impute_val=overall_mean_OQ_train)
test_data, test_labels = make_data_rectangular(test_OQ_list_new, df_width=best_width, impute_val=overall_mean_OQ_train)


ridge_model.fit(train_data, train_labels)
predictions = ridge_model.predict(test_data)
abs_errors = np.abs(predictions-test_labels)

In [ ]:
print(ridge_model.coef_)
print(ridge_model.intercept_)

In [ ]:
print(f"MSE: {np.mean(abs_errors**2)}")
print(f"MAE: {np.mean(abs_errors)}")

### Credible Interval

In [ ]:
nonzero_errors = abs_errors.copy()
nonzero_errors[nonzero_errors==0]=REPLACE_VAL

with pm.Model() as model:
    mean = pm.Normal('mean', means_mean, sigma=np.sqrt(means_var))
    var = pm.Gamma('var', alpha=vars_a, beta=vars_b)
    y = pm.Gamma('absolute error', alpha=mean**2/var, beta=mean/var, observed=nonzero_errors)
    trace=pm.sample(25000)

all_mean_draws = np.concatenate(np.array(trace['posterior']['mean']))
cutoff_idx = int(.025*len(all_mean_draws))
lower_bound = np.sort(all_mean_draws)[cutoff_idx]
upper_bound = np.sort(all_mean_draws)[-cutoff_idx-1]
print(f"{lower_bound}-{upper_bound}")

# Random Forests

## Define class

In [ ]:
class WidthRF(RandomForestRegressor):
    def __init__(self, df_width=1, n_estimators=100, criterion='squared_error', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=1.0, max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None):

        super().__init__(n_estimators=n_estimators, criterion=criterion, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, min_weight_fraction_leaf=min_weight_fraction_leaf, max_features=max_features, max_leaf_nodes=max_leaf_nodes, min_impurity_decrease=min_impurity_decrease, bootstrap=bootstrap, oob_score=oob_score, n_jobs=n_jobs, random_state=random_state, verbose=verbose, warm_start=warm_start, ccp_alpha=ccp_alpha, max_samples=max_samples)
        self.df_width = df_width

    def fit(self, X, y, sample_weight=None): #y is a necessary dummy
        #Assume that X is actually a set of variable-length time series, which we turn into data and labels
        mean = np.mean(np.concatenate(X))
        self.mean_val = mean
        train_data, train_labels = make_data_rectangular(X, df_width=self.df_width, impute_val=mean)
        super().fit(train_data, train_labels, sample_weight=sample_weight)

    def predict(self, X):
        #Note: If things aren't all already df_width in width,
        #   this will produce a different number of outputs than rows in X!!
        val_data, val_labels = make_data_rectangular(X, df_width=self.df_width, impute_val=self.mean_val)
        return super().predict(val_data)
    
    def score(self, X, y, sample_weight = None):
        #Overriding to use MSE (or negative MSE)
        val_data, val_labels = make_data_rectangular(X, df_width=self.df_width, impute_val=self.mean_val)
        predictions = super().predict(val_data)
        return -1*mean_squared_error(predictions, val_labels)

## Grid Search

In [ ]:
forest_full_data = WidthRF()

param_grid = {'n_estimators':[75,100,125],
                    'max_depth':[None, 5,10,20],
                    'min_samples_leaf':[1,2,4],
                    'max_features':['sqrt',None,0.5],
                    'df_width':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20],
                    'random_state':[64]}

grid_searcher = GridSearchCV(forest_full_data, param_grid, cv=3, verbose=2)

grid_searcher.fit(train_OQ_list_new, np.zeros(len(train_OQ_list_new)))

In [ ]:
print(grid_searcher.best_params_)
params_dict = {key:grid_searcher.best_params_[key] for key in grid_searcher.best_params_.keys() if key != 'df_width'}
best_width = grid_searcher.best_params_['df_width']

## Results

### Error, MSE, MAE

In [ ]:
np.random.seed(13)
rf_model = RandomForestRegressor(**params_dict)

train_data, train_labels = make_data_rectangular(train_OQ_list_new, df_width=best_width, impute_val=overall_mean_OQ_train)
rf_model.fit(train_data, train_labels)

val_data, val_labels = make_data_rectangular(val_OQ_list_new, df_width=best_width, impute_val=overall_mean_OQ_train)
predictions = rf_model.predict(val_data)

abs_errors = np.abs(predictions-test_labels)

In [ ]:
print(rf_model.feature_importances_)

In [ ]:
print(f"MSE: {np.mean(abs_errors**2)}")
print(f"MAE: {np.mean(abs_errors)}")

### Credible intervals

In [ ]:
nonzero_errors = abs_errors.copy()
nonzero_errors[nonzero_errors==0]=REPLACE_VAL

with pm.Model() as model:
    mean = pm.Normal('mean', means_mean, sigma=np.sqrt(means_var))
    var = pm.Gamma('var', alpha=vars_a, beta=vars_b)
    y = pm.Gamma('absolute error', alpha=mean**2/var, beta=mean/var, observed=nonzero_errors)
    trace=pm.sample(25000)
    
all_mean_draws = np.concatenate(np.array(trace['posterior']['mean']))
cutoff_idx = int(.025*len(all_mean_draws))
lower_bound = np.sort(all_mean_draws)[cutoff_idx]
upper_bound = np.sort(all_mean_draws)[-cutoff_idx-1]
print(f"{lower_bound}-{upper_bound}")

# Therapist predictions

## Sample test data for therapists

In [ ]:
with open('test_data_new.pkl', 'rb') as file:
    test_data_df = pickle.Unpickler(file=file).load()

master_df_2_test = pd.read_csv('master_df_2_cu003_thesis_hiddenClients.csv')

In [ ]:
#120 for each three therapists (one unused)
np.random.seed(499)
observation_indices = np.random.choice(len(test_data_df), size=360, replace=False)
sub_df = test_data_df.iloc[observation_indices].reset_index()[['Session','Previous', 'Next']]

In [ ]:
#Record which sessions, and where, are being given to therapists so we can link them to the true values later
rows_list = []
num_timeseries_per_therapist = 120
for therapist_num in range(3):
    for row_idx in range(120):
        row = sub_df.iloc[therapist_num*num_timeseries_per_therapist + row_idx]
        rows_list.append({'Study Therapist Index': therapist_num, 'Observation Index': row_idx, 'Session ID': row['Session'], '# of Previous OQs': len(row['Previous'])})

therapist_match_df = pd.DataFrame(rows_list)
therapist_match_df.to_csv('data_linking_info.csv')

In [ ]:
#These won't be reported directly, either for anonymity (not reported at all) or because their information is captured elsewhere
exclude_cols = ['PatientID', 'TherapistID', 'StartDate', 'EndDate', 'SessionID', 'NumOfAttended', 'Crisis', 'notedate', 'DateDifference', 'Date of Birth', 'Freshman / First-year', 'Sophomore', 'Junior', 'Senior', 'Graduate / professional degree student', 'Military Stress']

def detransform_features(features_series):
    #Takes in a pandas series of features from master_frame_2 and turns the numbers back into verbal responses
    #Well, in particular, it returns a long string with printed feature: answer pairs
    #More or less undoes what the data cleaning does to parse the patient info file

    def isnull(val):
        if val is None:
            return True
        if type(val) == float or isinstance(val, np.floating):
            return math.isnan(val)
        return False
    
    #Define functions to parse the most common answer types

    def get_bool_result(col_name):
        input = features_series[col_name]
        return "No response" if isnull(input) else ("Yes" if input else "No")
    
    def get_stress_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        try:
            assert type(input) == int
        except:
            input=int(input)
        assert input < 5 and input >= 0
        if not input:
            amount_str = "Never"
        elif input==1:
            amount_str = "Rarely"
        elif input==2:
            amount_str = "Sometimes"
        elif input==3:
            amount_str = "Often"
        else:
            amount_str = "Always"
        return amount_str + " stressful"
    
    def get_howmany_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        try:
            assert type(input) == int
        except:
            input=int(input)
        assert input < 5 and input >= 0
        if not input:
            ans_str = "Never"
        elif input==1:
            ans_str = "1 time"
        elif input==2:
            ans_str = "2-3 times"
        elif input==3:
            ans_str = "4-5 times"
        else:
            ans_str = "More than 5 times"
        return ans_str
    
    def get_sleep_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        try:
            assert type(input) == int
        except:
            input=int(input)
        assert input < 3 and input >= 0
        if not input:
            ans_str = "Rarely (0-1 nights)"
        elif input==1:
            ans_str = "Sometimes (2-3 nights)"
        else:
            ans_str = "Often (4+ nights)"
        return ans_str
    
    def get_support_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        try:
            assert type(input) == int
        except:
            input=int(input)
        assert input < 3 and input >= -2
        if not input:
            return "Neutral"
        if input==-2:
            pre_str = "Strongly dis"
        elif input==-1:
            pre_str = "Somewhat dis"
        elif input==1:
            pre_str = "Somewhat "
        else:
            pre_str = "Strongly "
        return pre_str + "agree"
    
    def get_religion_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        try:
            assert type(input) == int
        except:
            input=int(input)
        assert input < 3 and input >= -2
        if not input:
            return "Neutral"
        if input == 1:
            return "Important"
        if input==-2:
            pre_str = "Very un"
        elif input==-1:
            pre_str = "Un"
        else:
            pre_str = "Very "
        return pre_str + "important"
    
    def get_int_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        return int(input)
    
    def get_social_media_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        try:
            assert type(input) == int
        except:
            input=int(input)
        assert input < 6 and input >= 1
        if input == 1:
            return "Mostly negative"
        if input==2:
            return "Somewhat negative"
        if input==3:
            return "Somewhat positive"
        if input == 4:
            return "Mostly positive"
        return "I do not use social media"
    
    def get_sex_result(col_name):
        input = features_series[col_name]
        if isnull(input):
            return "No response"
        return ("Female" if input else "Male")
    
    # Abbreviate or line-break long questions

    disability_q = "Are you registered, with the office for disability services on this campus, as having a documented and diagnosed disability?"
    formatted_disability_q = "Are you registered, with the office for disability services on this campus,\n\tas having a documented and diagnosed disability?"

    disability_codes = {
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Difficulty hearing': 'Difficulty hearing',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Difficulty seeing': 'Difficulty seeing',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Difficulty speaking or language impairment': 'Difficulty speaking or language impairment',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Mobility limitation/ orthopedic impairment': 'Mobility limitation/ orthopedic impairment',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Traumatic brain injury': 'Traumatic brain injury',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Specific learning disabilities': 'Specific learning disabilities',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: ADD or ADHD': 'ADD or ADHD',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Autism spectrum disorders': 'Autism spectrum disorder',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Cognitive difficulties or intellectual disability': 'Cognitive difficulties or intellectual disability',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Health impairment/ condition, including chronic conditions': 'Health impairment/ condition, including chronic conditions',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Psychological or psychiatric condition': 'Psychological or psychiatric condition',
        'If you selected, "Yes" for the previous question, please indicate which category of disability you are registered for (check all that apply) ANSWER: Other': 'Other'
    }

    trauma_codes = {
        'Please select the traumatic event(s) you have experienced ANSWER: Childhood physical abuse': 'Childhood physical abuse',
        'Please select the traumatic event(s) you have experienced ANSWER: Childhood sexual abuse': 'Childhood sexual abuse',
        'Please select the traumatic event(s) you have experienced ANSWER: Childhood emotional abuse': 'Childhood emotional abuse',
        'Please select the traumatic event(s) you have experienced ANSWER: Physical attack (e.g., mugged, beaten up, shot, stabbed, threatened with weapon)': 'Physical attack',
        'Please select the traumatic event(s) you have experienced ANSWER: Sexual violence (rape or attempted rape, sexually assaulted, stalked, abused by intimate partner, etc.)': 'Sexual violence',
        'Please select the traumatic event(s) you have experienced ANSWER: Military combat or war zone experiences': 'Military combat or war zone experiences',
        'Please select the traumatic event(s) you have experienced ANSWER: Kidnapped or taken hostage': 'Kidnapped or taken hostage',
        'Please select the traumatic event(s) you have experienced ANSWER: Serious accident, fire, or explosion (e.g., an industrial, farm, car, plane, or boating accident)': 'Serious accident, fire, or explosion',
        'Please select the traumatic event(s) you have experienced ANSWER: Terrorist attack': 'Terrorist attack',
        'Please select the traumatic event(s) you have experienced ANSWER: Near drowning': 'Near drowning',
        'Please select the traumatic event(s) you have experienced ANSWER: Diagnosed with life threatening illness': 'Diagnosed with life-threatening illness',
        'Please select the traumatic event(s) you have experienced ANSWER: Natural disaster (e.g., flood, quake, hurricane, etc.)': 'Natural disaster',
        'Please select the traumatic event(s) you have experienced ANSWER: Imprisonment or Torture': 'Imprisonment or torture',
        'Please select the traumatic event(s) you have experienced ANSWER: Animal attack': 'Animal attack',
        'Please select the traumatic event(s) you have experienced ANSWER: Other (please specify)': 'Other'
    }

    autism_q = "Have you been diagnosed with an autism-spectrum disorder or Asperger's Syndrome?"
    formatted_autism_q = "Have you been diagnosed with an autism-spectrum disorder\n\tor Asperger's Syndrome?"
    sleep_q_1 = "In the past week, about how many nights has it taken you more than half an hour to fall asleep?"
    formatted_sleep_q_1 = "In the past week, about how many nights has it taken\n\tyou more than half an hour to fall asleep?"
    sleep_q_2 = "In the past week, about how many nights have you woken during the night AND needed more than half an hour to fall back to sleep?"
    formatted_sleep_q_2 = "In the past week, about how many nights have you woken during\n\tthe night AND needed more than half an hour\n\tto fall back to sleep?"
    sleep_q_3 = "In the past two weeks, have you taken a substance to help with sleep? Please consider prescription medication, over the counter medication and supplements, and substances such as alcohol and marijuana."
    formatted_sleep_q_3 = "In the past two weeks, have you taken a substance to help\n\twith sleep? Please consider prescription medication, over the counter\n\tmedication and supplements, and substances such as\n\talcohol and marijuana."

    #More functions, to handle lists of true-falses where only the "true" answers are relevant
    def generate_disability_list():
        disabilities_list = []
        for col in disability_codes.keys():
            if features_series[col]:
                disabilities_list.append(disability_codes[col])
        return disabilities_list
    
    def generate_trauma_list():
        trauma_list = []
        for col in trauma_codes.keys():
            if features_series[col]:
                trauma_list.append(trauma_codes[col])
        return trauma_list
    
    def find_academic_year():
        for key in ['Freshman / First-year', 'Sophomore', 'Junior', 'Senior', 'Graduate / professional degree student']:
            if features_series[key]:
                return key
        return "No response"

    #Now: Link intake questions to something human-readable, separated mostly by order presented to therapists

    explanation_dict_part_1 = {
        'Female': 'Sex assigned at birth',
        'age': 'Age',
        'RacialMinority': 'Racial Minority',
        'International Student': 'International Student',
        'SexOrientationMinority': "Sexual Orientation Minority",
        'MarriedMale': 'Married Male',
        'NursingOrLaw': 'Nursing or Law Student',
        disability_q: formatted_disability_q
    }

    #Which disabilities here

    explanation_dict_part_2 = {
        autism_q: formatted_autism_q,
        'Homeless': 'Homeless',
        'ROTC': 'ROTC Member',
        'Military Service': 'Client Military Service'
    }

    #Break here to include military stress if they said yes to military service

    #All of the "recent" will be conditional on the preceding "how many"
    explanation_dict_part_3 = {
        'First Generation': 'First-Generation College Student',
        'Financial Stress Now': '\nCurrent level of financial stress',
        'Financial Stress Past': 'Level of financial stress while growing up',
        'AgnosticOrAtheist': 'Agnostic or Atheist',
        'ReligiousMinority': 'Religious Minority',
        'Religion Importance': 'Importance of religion to client',
        'Prior Counseling': 'Has the client ever gone to mental health counseling before?',
        'Prior Meds': 'Has the client ever taken medication for mental health reasons?',
        'Prior Hospitalization (How many)': '\nHow many times has the client been\n\thospitalized for mental health reasons?',
        'Prior Hospitalization (Recent)': 'Did the client indicate they had been\n\thospitalized for mental health reasons during the past year?',
        'Need to Reduce D&A (How many)': 'How many times has the client felt a\n\tneed to reduce drug and alcohol use?',
        'Need to Reduce D&A (Recent)': 'Did the client indicate they had felt a\n\tneed to reduce drug and alcohol use during the past year?',
        'Others Concern Alcohol (How many)': "How many times have others expressed to the client\n\tconcern about the client's drug and alcohol use?",
        'Others Concern Alcohol (Recent)': "Did the client indicate that others expressed concern\n\tabout their drug and alcohol use during the past year?",
        'Prior D&A Treatment (How many)': "How many times has the client received treatment\n\tfor their drug and alcohol use?",
        'Prior D&A Treatment (Recent)': "Did the client indicate that they received treatment\n\tfor their drug and alcohol use in the past year?",
        'Self-Injury (How many)': "How many times has the client injured themselves\n\twithout the intent to cause death?",
        'Self-Injury (Recent)': "Did the client indicate they had injured themselves\n\twithout the intent to cause death in the past year?",
        'Considered Suicide (How many)': "How many times has the client considered suicide?",
        'Considered Suicide (Recent)': "Did the client indicate they had considered suicide\n\tin the past year?",
        'Suicide Attempt (How many)': "How many times has the client attempted suicide?",
        'Suicide Attempt (Recent)': "Did the client indicate they had\n\tattempted suicide in the past year?",
        'Considered Harming (How many)': "How many times has the client considered\n\tcausing serious physical harm to another person?",
        'Considered Harming (Recent)': "Did the client indicate they had considered\n\tcausing serious physical harm to another person in the past year?",
        'Harmed Another (How many)': "How many times has the client intentionally injured another?",
        'Harmed Another (Recent)': "Did the client indicate they had intentionally\n\tinjured another in the past year?",
        'Unwanted Sexual Exp. (How many)': "How many times has the client had an unwanted\n\tsexual experience? (E.g. they were passed out,\n\tdrugged, threatened, etc.)",
        'Unwanted Sexual Exp. (Recent)': "Did the client indicate they had an unwanted\n\tsexual experience in the past year?",
        'Harassment/Abuse (How many)': "How many times has the client experienced harassing,\n\tcontrolling, or abusive behavior from another?",
        'Harassment/Abuse (Recent)': "Did the client indicate they had experienced harassing,\n\tcontrolling, or abusive behavior from another in the past year?",
        'PTSD Experience (How many)': "How many times has the client experienced a traumatic\n\tevent that cause a feeling of fear, helplessness, or horror?",
        'PTSD Experience (Recent)': "Did the client indicate they had experienced a traumatic\n\tevent causing fear, helplessness, or horror in the past year?"
    }

    explanation_dict_part_4 = {
        'Family Support': '\nDoes the client agree that they experience family support?',
        'Social Support': 'Does the client agree that they experience social support?',
        sleep_q_1: formatted_sleep_q_1,
        sleep_q_2: formatted_sleep_q_2,
        sleep_q_3: formatted_sleep_q_3,
        'Academics': 'What is the impact of social media on your academics?',
        'Social life/relationships': 'What is the impact of social media on\n\tyour social life / relationship?',
        'Emotional well-being': 'What is the impact of social media on your\n\temotional well-being?',
        'Confusion about religious beliefs or values': '\nConfusion about religious beliefs or values',
        'Gender, ethnic, or racial discrimination': 'Gender, ethnic, or racial discrimination',
        'Perfectionism': 'Perfectionism',
        'Physical health problems (headaches, GI trouble)': 'Physical health problems (headaches, GI trouble)',
        'Sexual concerns': 'Sexual concerns',
        'Sexual orientation or identity': 'Sexual orientation or identity',
        'Pornography': 'Pornography'
    }


    all_explained_keys = []
    for key in explanation_dict_part_1.keys():
        all_explained_keys.append(key)
    for key in explanation_dict_part_2.keys():
        all_explained_keys.append(key)
    for key in explanation_dict_part_3.keys():
        all_explained_keys.append(key)
    for key in explanation_dict_part_4.keys():
        all_explained_keys.append(key)
    for key in disability_codes.keys():
        all_explained_keys.append(key)
    for key in trauma_codes.keys():
        all_explained_keys.append(key)

    explanation_dict_part_5 = {col:col for col in features_series.index if (col not in all_explained_keys and col not in exclude_cols)}

    #List features by which function is used to parse them
    int_parser_features = ['age']
    sex_parser_features = ['Female']
    bool_parser_features = ['RacialMinority', 'International Student', 'SexOrientationMinority', 'MarriedMale', 'NursingOrLaw', disability_q, autism_q, 'Homeless', 'ROTC', 'Military Service', 'Military Stress', 'First Generation', 'AgnosticOrAtheist', 'ReligiousMinority', 'Prior Counseling', 'Prior Meds', 'Prior Hospitalization (Recent)', 'Need to Reduce D&A (Recent)', 'Others Concern Alcohol (Recent)', 'Prior D&A Treatment (Recent)', 'Self-Injury (Recent)', 'Considered Suicide (Recent)', 'Suicide Attempt (Recent)', 'Considered Harming (Recent)', 'Harmed Another (Recent)', 'Unwanted Sexual Exp. (Recent)', 'Harassment/Abuse (Recent)', 'PTSD Experience (Recent)']
    stress_parser_features = ['Financial Stress Now', 'Financial Stress Past', 'Confusion about religious beliefs or values', 'Gender, ethnic, or racial discrimination', 'Perfectionism', 'Physical health problems (headaches, GI trouble)', 'Sexual concerns', 'Sexual orientation or identity', 'Pornography']
    religion_parser_features = ['Religion Importance']
    howmany_parser_features = ['Prior Hospitalization (How many)', 'Need to Reduce D&A (How many)', 'Others Concern Alcohol (How many)', 'Prior D&A Treatment (How many)', 'Self-Injury (How many)', 'Considered Suicide (How many)', 'Suicide Attempt (How many)', 'Considered Harming (How many)', 'Harmed Another (How many)', 'Unwanted Sexual Exp. (How many)', 'Harassment/Abuse (How many)', 'PTSD Experience (How many)']
    support_parser_features = ['Family Support', 'Social Support']
    sleep_parser_features = [sleep_q_1, sleep_q_2, sleep_q_3]
    social_media_parser_features = ['Academics', 'Social life/relationships', 'Emotional well-being']

    all_parsed_features = int_parser_features + sex_parser_features + bool_parser_features + stress_parser_features + religion_parser_features + howmany_parser_features + support_parser_features + sleep_parser_features + social_media_parser_features
    int_parser_features = [col for col in features_series.index if (col not in all_parsed_features and col not in exclude_cols)] + int_parser_features

    parser_dict = {col: get_int_result for col in int_parser_features}

    #Set up parsing functions as a dictionary for easier coding
    for col in sex_parser_features:
        parser_dict[col] = get_sex_result
    for col in bool_parser_features:
        parser_dict[col] = get_bool_result
    for col in stress_parser_features:
        parser_dict[col] = get_stress_result
    for col in religion_parser_features:
        parser_dict[col] = get_religion_result
    for col in howmany_parser_features:
        parser_dict[col] = get_howmany_result
    for col in support_parser_features:
        parser_dict[col] = get_support_result
    for col in sleep_parser_features:
        parser_dict[col] = get_sleep_result
    for col in social_media_parser_features:
        parser_dict[col] = get_social_media_result

    result_string = "Below is a list of question-answer pairs and other information about\nthe client's concerns and demographics. The format of the information varies,\nbut 'you', 'I', and 'the client' all refer to the client.\n\n"

    #Put all the answers together into one big string
    #Academic status first
    result_string += f"Academic status: {find_academic_year()}\n"
    for key in explanation_dict_part_1.keys():
        result_string += f"{explanation_dict_part_1[key]}: {parser_dict[key](key)}\n"
    #Disabilities
    if features_series[disability_q]:
        result_string += "Registered categories of disabilities: "
        for disability in generate_disability_list():
            result_string += disability+", "
        result_string = result_string[:-2] + "\n"
    
    for key in explanation_dict_part_2.keys():
        result_string += f"{explanation_dict_part_2[key]}: {parser_dict[key](key)}\n"
    if features_series['Military Service']:
        result_string += f"Military service included stressful or traumatic experience that continues to bother the client: {get_bool_result('Military Stress')}\n"

    for key in explanation_dict_part_3.keys():
        if '(Recent)' not in key:
            result_string += f"{explanation_dict_part_3[key]}: {parser_dict[key](key)}\n"
        else:
            root_question = key[:-7]+'How many)'
            if features_series[root_question]:
                result_string += f"{explanation_dict_part_3[key]}: {parser_dict[key](key)}\n"
    
    if features_series['PTSD Experience (How many)']:
        result_string += "Types of past traumatic events:\n\t"
        for trauma in generate_trauma_list():
            result_string += trauma+", "
        result_string = result_string[:-2] + "\n"

    for key in explanation_dict_part_4.keys():
        result_string += f"{explanation_dict_part_4[key]}: {parser_dict[key](key)}\n"

    result_string+="\n--Note: The remaining questions come from the CCAPS,\n\twhich ranks answers on a 5-point scale from\n\t0 (not at all like me) and 4 (extremely like me).--\n\n"
    for key in explanation_dict_part_5.keys():
        result_string += f"{explanation_dict_part_5[key]}: {parser_dict[key](key)}\n"
    
    return result_string

In [ ]:
#Write the OQ data, and intake if found, to a file to send to a therapist to predict on
def create_prediction_file(data, save_path):
    #data will be a row from sub_df or something like it
    #save_path will tell the script the folder to save a file to

    with open(save_path, 'w') as file:
        #First, write previous OQ scores
        file.write(f'Previous OQ scores: ')
        string_to_add = ""
        for val in data['Previous']:
            string_to_add += str(int(val)) + ", "
        string_to_add = string_to_add[:-2] + "\n\n\n"
        file.write(string_to_add)

        #If there's intake data, add that too
        if data['Session'] in list(master_df_2_test['SessionID']):
            try:
                assert len(master_df_2_test[master_df_2_test['SessionID']==data['Session']])==1
            except:
                raise
            idx = master_df_2_test[master_df_2_test['SessionID']==data['Session']].index[0]
            file.write(detransform_features(master_df_2_test.iloc[idx])+"\n\n")
        file.write("Prediction: ")

In [ ]:
#Create the files
for therapist_num in range(3):
    for row_idx in range(120):
        create_prediction_file(sub_df.iloc[therapist_num*num_timeseries_per_therapist + row_idx], f'therapist_{therapist_num}/{row_idx+1}.txt')

## Parse Responses

In [ ]:
#Load data and set up information needed to get therapist predictions from special file structure
therapist_match_df = pd.read_csv('data_linking_info.csv')
therapist_match_dict = {}
for row_idx in range(len(therapist_match_df)):
    row = therapist_match_df.iloc[row_idx]
    therapist_match_dict[(row['Study Therapist Index'], row['Observation Index']+1)] = (row['Session ID'], row['# of Previous OQs'])
    #I hope that works
with open('test_data_new.pkl', 'rb') as file:
    test_data_df = pickle.Unpickler(file=file).load()

subfolder_dict = {}
for i in range(1,21):
    subfolder_dict[i] = "1-20"
for i in range(21,41):
    subfolder_dict[i] = "21-40"
for i in range(41,61):
    subfolder_dict[i] = "41-60"
for i in range(61,81):
    subfolder_dict[i] = "61-80"
for i in range(81,101):
    subfolder_dict[i] = "81-100"
for i in range(101,121):
    subfolder_dict[i] = "101-120"

In [ ]:
#Given a session number and how far into it we are, get the real next value
def get_true_val(session, num_prev):
    sub_df = test_data_df[test_data_df['Session']==session]
    ans_rows_list = [sub_df.iloc[i] for i in range(len(sub_df)) if len(sub_df.iloc[i]['Previous'])==num_prev]
    assert len(ans_rows_list)==1
    return int(ans_rows_list[0]['Next'])

In [ ]:
#This function actually gets therapist prediction data out from the specialized files they're in, and gets the real values
# (they were returned several different ways)

def read_therapist_predictions(folder_name, therapist_idx, mode='text', input_filename='', index_col = 'vignette', data_col = 'predicted OQ'): #therapist_idx is within the study - should be 0, 1, or 2
        #filename argument only used if mode==csv
    assert mode in ['text', 'csv']
    if mode=='csv' and (input_filename=='' or input_filename is None):
        raise ValueError("Need a specific filename for csv parsing")
    

    true_vals = np.zeros(120)
    predictions = np.zeros(120)

    if mode=='csv':
        filename=folder_name+'/'+input_filename
        predictions_df = pd.read_csv(filename).set_index(index_col)

    for idx in range(1,121):
        if mode=='text':
            filename = folder_name+'/'+subfolder_dict[idx]+'/'+str(idx)+".txt"
            with open(filename, 'r') as file:
                last_line = file.readlines()[-1]
                assert last_line[:11] == "Prediction:"
                predictions[idx-1] = int(last_line[11:].strip())
                session, num_prev = therapist_match_dict[(therapist_idx, idx)]
                true_vals[idx-1] = get_true_val(session, num_prev)

        elif mode=='csv':
            predictions[idx-1] = int(predictions_df.loc[idx][data_col])
            session, num_prev = therapist_match_dict[(therapist_idx, idx)]
            true_vals[idx-1] = get_true_val(session, num_prev)
    return true_vals, predictions

In [ ]:
#Get the predictions and true values for therapist 2
true_vals_2, predictions_2 = read_therapist_predictions('therapist_2_predictions', 2)

In [ ]:
#Load predictions and true values in the time series for the other therapist, therapist 0
true_vals_0, predictions_0 = read_therapist_predictions('therapist_0_predictions',0,'csv','therapist_0_predictions.csv')

## Get results

In [ ]:
abs_errors_0 = np.abs(true_vals_0-predictions_0)
abs_errors_2 = np.abs(true_vals_2-predictions_2)
abs_errors_pooled = np.concatenate([abs_errors_0, abs_errors_2])

### Therapist 0

In [ ]:
#MSE and MAE
print(f"Therapist 0 MSE: {np.mean(abs_errors_0**2)}")
print(f"Therapist 0 MAE: {np.mean(abs_errors_0)}")

In [ ]:
#Credible interval
nonzero_errors_0 = abs_errors_0.copy()
nonzero_errors_0[nonzero_errors_0==0]=REPLACE_VAL

with pm.Model() as model:
    mean = pm.Normal('mean', means_mean, sigma=np.sqrt(means_var))
    var = pm.Gamma('var', alpha=vars_a, beta=vars_b)
    y = pm.Gamma('absolute error', alpha=mean**2/var, beta=mean/var, observed=nonzero_errors_0)
    trace=pm.sample(25000)

all_mean_draws = np.concatenate(np.array(trace['posterior']['mean']))
cutoff_idx = int(.025*len(all_mean_draws))
lower_bound = np.sort(all_mean_draws)[cutoff_idx]
upper_bound = np.sort(all_mean_draws)[-cutoff_idx-1]
print(f"{lower_bound}-{upper_bound}")

### Therapist 2

In [ ]:
#MSE and MAE
print(f"Therapist 2 MSE: {np.mean(abs_errors_2**2)}")
print(f"Therapist 2 MAE: {np.mean(abs_errors_2)}")

In [ ]:
#Credible interval
nonzero_errors_2 = abs_errors_2.copy()
nonzero_errors_2[nonzero_errors_2==0]=REPLACE_VAL

with pm.Model() as model:
    mean = pm.Normal('mean', means_mean, sigma=np.sqrt(means_var))
    var = pm.Gamma('var', alpha=vars_a, beta=vars_b)
    y = pm.Gamma('absolute error', alpha=mean**2/var, beta=mean/var, observed=nonzero_errors_2)
    trace=pm.sample(25000)
    
all_mean_draws = np.concatenate(np.array(trace['posterior']['mean']))
cutoff_idx = int(.025*len(all_mean_draws))
lower_bound = np.sort(all_mean_draws)[cutoff_idx]
upper_bound = np.sort(all_mean_draws)[-cutoff_idx-1]
print(f"{lower_bound}-{upper_bound}")

### Pooled therapists

In [ ]:
#MSE and MAE
print(f"Pooled therapist MSE: {np.mean(abs_errors_pooled**2)}")
print(f"Pooled therapist MAE: {np.mean(abs_errors_pooled)}")

In [ ]:
#Credible interval
nonzero_errors_pooled = abs_errors_pooled.copy()
nonzero_errors_pooled[nonzero_errors_pooled==0]=REPLACE_VAL

with pm.Model() as model:
    mean = pm.Normal('mean', means_mean, sigma=np.sqrt(means_var))
    var = pm.Gamma('var', alpha=vars_a, beta=vars_b)
    y = pm.Gamma('absolute error', alpha=mean**2/var, beta=mean/var, observed=nonzero_errors_pooled)
    trace=pm.sample(25000)
    
all_mean_draws = np.concatenate(np.array(trace['posterior']['mean']))
cutoff_idx = int(.025*len(all_mean_draws))
lower_bound = np.sort(all_mean_draws)[cutoff_idx]
upper_bound = np.sort(all_mean_draws)[-cutoff_idx-1]
print(f"{lower_bound}-{upper_bound}")

# LSTMs

Most of the code relating to LSTMs was performed though a high-performance computing service using shell scripts and a command-line interface. The necessary files to reproduce the process are in this repo; The process is followed by running the following commands, in order, in an appropriate environment. Note that some files have placeholder names instead of real folders, virtual environments, etc.
#TODO Add what's in the environment

1. python create_config_files.py
1. sbatch gridsearch_new_cleaning.sh
1. Use the notebook parse_gridsearch_outputs.ipynb to determine the hyperparameters, and enter them into new_cleaning_training.py
1. sbatch train_new_cleaning.sh
1. Select the saved model with the best validation-set accuracy out of placeholder folder 3 and save it somewhere accessible to this notebook, then use the below cells to get results

In [ ]:
with open('PLACEHOLDER_FOLDER_3_or_4/path/to/best/model', 'rb') as file:
        model = torch.load(file, map_location=torch.device('cpu'))

In [ ]:
#Best model's df_width
MODEL_WIDTH = 3

In [ ]:
test_data, test_labels = make_data_rectangular(test_OQ_list_new, df_width = MODEL_WIDTH, impute_val=overall_mean_OQ_train)

X_test, y_test = torch.tensor(test_data).float().unsqueeze(2), torch.tensor(test_labels).float().unsqueeze(1).unsqueeze(2)

y_pred = model(X_test)[:,-1,:].unsqueeze(1)
abs_errors = np.abs(np.array(y_test).squeeze()-np.array(y_pred.detach().squeeze()))

In [ ]:
print(f"MSE: {np.mean(abs_errors**2)}")
print(f"MAE: {np.mean(abs_errors)}")

In [ ]:
nonzero_errors = abs_errors.copy()
nonzero_errors[nonzero_errors==0]=REPLACE_VAL

with pm.Model() as model:
    mean = pm.Normal('mean', means_mean, sigma=np.sqrt(means_var))
    var = pm.Gamma('var', alpha=vars_a, beta=vars_b)
    y = pm.Gamma('absolute error', alpha=mean**2/var, beta=mean/var, observed=nonzero_errors)
    trace=pm.sample(25000)
    
all_mean_draws = np.concatenate(np.array(trace['posterior']['mean']))
cutoff_idx = int(.025*len(all_mean_draws))
lower_bound = np.sort(all_mean_draws)[cutoff_idx]
upper_bound = np.sort(all_mean_draws)[-cutoff_idx-1]
print(f"{lower_bound}-{upper_bound}")